[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AIladin/diploma-script/blob/main/CRRDefaultModel.ipynb)

In [1]:
!git clone https://github.com/AIladin/diploma-script.git
%cd diploma-script

Cloning into 'diploma-script'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 2), reused 10 (delta 2), pack-reused 0
Unpacking objects: 100% (10/10), done.
/content/diploma-script


In [2]:
import numpy as np
from main import CRRModel, CRRModelCalculator

# Initialization

In [3]:
model = CRRModel(
    s_0 = 100,
    b_0 = 50,
    a = -0.5,
    b = 0.5,
    r = 0.1,
)

In [4]:
K = 10

def payment_function(x: np.ndarray) -> np.ndarray:
    return (x - K).clip(min=0)

In [5]:
N = 21

calculator = CRRModelCalculator(model, payment_function, N)

# Tests

## Fair price $C_N$

In [6]:
calculator.get_fair_price()

array([1.17932828])

## $\gamma_n$, $\beta_n$, $X_n^\pi$

In [9]:
%time
for n in range(1, N + 1):
    print("Timestep: ", n)
    print("Beta: ", calculator.get_beta(n))
    print("Gamma ", calculator.get_gamma(n))
    print("Discounted capital", calculator.get_discounted_capital(n))
    print()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
Timestep:  1
Beta:  [-0.00890745]
Gamma  [0.01624701]
Discounted capital [0.64738081 2.27208156]

Timestep:  2
Beta:  [-0.00512844 -0.01457597]
Gamma  [0.0185889  0.02049173]
Discounted capital [0.34034094 1.2697858  1.2697858  4.34354559]

Timestep:  3
Beta:  [-0.00286679 -0.00852091 -0.00852091 -0.02365856]
Gamma  [0.02055127 0.02380401 0.02380401 0.02566617]
Discounted capital [0.16886233 0.68264409 0.68264409 2.46794482 0.68264409 2.46794482
 2.46794482 8.24283315]

Timestep:  4
Beta:  [-0.00152744 -0.00487582 -0.00487582 -0.01398854 -0.00487582 -0.01398854
 -0.01398854 -0.03816358]
Gamma  [0.02164107 0.02685679 0.02685679 0.03021229 0.02685679 0.03021229
 0.03021229 0.0319485 ]
Discounted capital [ 0.0775432   0.34805661  0.34805661  1.35518634  0.34805661  1.35518634
  1.35518634  4.75406875  0.34805661  1.35518634  1.35518634  4.75406875
  1.35518634  4.75406875  4.75406875 15.53668803]

Timestep:  5
Beta:  [-0.00075

## Payment function plots

In [10]:
import plotly.graph_objects as go
import plotly.express as px

In [11]:
x = list(range(N + 1))
lower_bound = [np.min(payment_function(model.stock_price_evolution(n))) for n in range(N + 1)]
mean = [np.mean(payment_function(model.stock_price_evolution(n))) for n in range(N + 1)]
upper_bound = [np.max(payment_function(model.stock_price_evolution(n))) for n in range(N + 1)]

fig = go.Figure()

fig.update_layout(title="Payment function evolution", xaxis_title="$n$", yaxis_title="$f_n(S_n) = (S_n - K)^+$", height=600)

fig.add_trace(go.Scatter(x=x + x[::-1], y=upper_bound + lower_bound[::-1], fill="toself",  name="Payment function bounds."))
fig.add_trace(go.Scatter(x=x, y=mean, name="Mean payment function value."))
fig.update_traces(mode="lines")
fig.show()

In [13]:
n = 7
fig = go.Figure(data=[go.Histogram(x=payment_function(model.stock_price_evolution(n)))])
fig.update_layout(title="Payment function distribution on each step", xaxis_title=f"$f_{n}(S_{n}) = (S_{n} - K)^+$", height=600)
fig.show()